In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib qt

In [2]:
from ICARUS.Database import DB
from ICARUS.Database import EXTERNAL_DB
DB.load_data()

# Load Database

In [3]:
db = DB.foils_db

from ICARUS.Computation.Solvers.XFLR5.polars import read_polars_2d
read_polars_2d(EXTERNAL_DB)

airfoils = db.data

# index = int(input("Select an airfoil: "))
index = 312
airfoil_names: list[str] = list(airfoils.keys())
print(f"We have a total of {len(airfoil_names)} airfoils")

We have a total of 745 airfoils


In [4]:
naca_airfoils = [airfoil_name for airfoil_name in airfoil_names if airfoil_name.startswith("NACA")]
print(f"We have a total of {len(naca_airfoils)} NACA airfoils")

clarky_airfoils = [airfoil_name for airfoil_name in airfoil_names if airfoil_name.startswith("C")]
print(f"We have a total of {len(clarky_airfoils)} Clark Y airfoils")

s_airfoils = [airfoil_name for airfoil_name in airfoil_names if airfoil_name.startswith("S")]
print(f"We have a total of {len(s_airfoils)} S airfoils")

a_airfoils = [airfoil_name for airfoil_name in airfoil_names if airfoil_name.startswith("A")]
print(f"We have a total of {len(a_airfoils)} A airfoils")

We have a total of 107 NACA airfoils
We have a total of 9 Clark Y airfoils
We have a total of 138 S airfoils
We have a total of 34 A airfoils


# Airfoil Polars

In [5]:
from ICARUS.Visualization.airfoil.db_polars import plot_airfoils_polars

In [6]:
axs, fig = plot_airfoils_polars(
    airfoil_names= s_airfoils,
    solvers = ["Xfoil" ],
    reynolds= 1e6,
    plots = [["AoA", "CL"], ["AoA", "CD"], ["AoA", "Cm"], ["AoA", "CL/CD"]],
    size=(10,9),
)
# axs[0].axvline(x=1.1, color='b', linestyle='-')
# axs[0].axhline(y=1.52, color='b', linestyle='-')

In [8]:
from ICARUS.Airfoils.airfoil import Airfoil
from ICARUS.Airfoils.airfoil_polars import Polars
import matplotlib.pyplot as plt
import numpy as np
s_polars = [polar for k,polar in db.polars.items() if k.startswith("D")]
for p in s_polars[:10]:
    try:
        polar: Polars = p['Xfoil']
        reyn_min = polar.reynolds_keys[0]
        reyn_max = polar.reynolds_keys[-1]
        # Create 2 subplots and unpack the output array immediately
        fig, axs = plt.subplots(2, 2, figsize=(10, 9))

        fig.suptitle(f"{polar.name} Polars")
        polar.plot_reynolds_cl_curve(reyn_min, fig, axs[0,0])
        polar.plot_reynolds_cl_curve(reyn_max, fig, axs[0,0])
        polar.plot_reynolds_cd_curve(reyn_min, fig, axs[0,1])
        polar.plot_reynolds_cd_curve(reyn_max, fig, axs[0,1])
        polar.plot_reynolds_cl_over_cd_curve(reyn_min, fig, axs[1,0])
        polar.plot_reynolds_cl_over_cd_curve(reyn_max, fig, axs[1,0])

        # Plot the airfoil on the second subplot
        airfoil: Airfoil = DB.foils_db.airfoils[polar.name]
        airfoil.plot(ax = axs[1,1], camber=True, max_thickness=True)

        # Add text for the slopes
        axs[0,0].text(0.7, 0.2, f"Cl slope at {float(reyn_min):.1e}: {polar.get_reynolds_cl_slope(reyn_min)* 180/np.pi:.3f} ", fontsize=12, transform=axs[0,0].transAxes)
        axs[0,0].text(0.7, 0.1, f"Cl slope at {float(reyn_max):.1e}: {polar.get_reynolds_cl_slope(reyn_max)* 180/np.pi:.3f} ", fontsize=12, transform=axs[0,0].transAxes)
        fig.tight_layout()
    except Exception as e:
        print(e)
        continue

# Select Reynolds number 

In [9]:
reynolds = db.polars[airfoil_names[0]]["Xfoil"].reynolds_keys
reynolds

['1.001e06',
 '1.122e06',
 '1.531e05',
 '1.756e05',
 '2.742e05',
 '3.205e04',
 '3.953e05',
 '4.109e05',
 '5.164e05',
 '6.375e05',
 '7.501e04',
 '7.585e05',
 '8.796e05',
 '9.615e05']

In [10]:
reynolds = db.polars[airfoil_names[0]]["Xfoil"].reynolds_keys

if reynolds is not None:
    print(f"For NACA {airfoil_names[0][4:]} these Reynolds were Computed:")
    print(*[f"{i}:\t{reyn}\n" for i,reyn in enumerate(reynolds)])
    index = int(input("Select Reynolds: "))
    reyn_to_plot = reynolds[index]
else:
    raise ValueError(f"Reynolds for {airfoil_names[0]} were not computed")

For NACA  these Reynolds were Computed:
0:	1.001e06
 1:	1.122e06
 2:	1.531e05
 3:	1.756e05
 4:	2.742e05
 5:	3.205e04
 6:	3.953e05
 7:	4.109e05
 8:	5.164e05
 9:	6.375e05
 10:	7.501e04
 11:	7.585e05
 12:	8.796e05
 13:	9.615e05



ValueError: invalid literal for int() with base 10: ''

# Plot Reynolds

In [ ]:
from ICARUS.Visualization.airfoil.airfoil_reynolds import plot_airfoil_reynolds

In [ ]:
print(f"NACA {airfoil_names[0][4:]} at Reynolds {reyn_to_plot} polars are:")
axs, fig = plot_airfoil_reynolds(
    airfoil_names[0],
    reynolds[0],
    ["All"],
    plots = [["AoA", "Cm"], ["AoA", "CD"]],
    size=(10,9),
)
# axs[0].axvline(x=1.1, color='b', linestyle='-')
# axs[0].axhline(y=1.52, color='b', linestyle='-')